In [1]:
import psutil
import time
import pandas as pd
import numpy as np
from pyccata.core.configuration import Configuration as Con
from pyccata.core.extractor import PartitionRunner
from pyccata.core.managers.thread import ThreadManager
from pyccata.core.extractor import DataThreader
from pyccata.bioinformatics.resources import BedFileItem

In [2]:
(((psutil.virtual_memory().available / 1024 ) / 1024) / 1024)

33.54029846191406

In [3]:
configuration = Con(filename='/home/mproffitt/notebooks/config.json', dont_parse=True)

[DEBUG] Loading singleton configuration with filename /home/mproffitt/notebooks/config.json
[DEBUG] Checking for config file in '/home/mproffitt/notebooks'
[DEBUG] Using configuration from '/home/mproffitt/notebooks/config.json'.
[INFO] Loading manager class 'csv'
[INFO] Loading reporting class 'null'
[DEBUG] Loading singleton replacements


In [4]:
df1 = pd.read_csv('/home/mproffitt/final-peaks/run-flattened-hdacs/datafiles/HDac1.bed', delimiter='\t', header=None)
df2 = pd.read_csv('/home/mproffitt/final-peaks/run-flattened-hdacs/datafiles/HDac2.bed', delimiter='\t', header=None)
df1.columns = BedFileItem().keys()
df2.columns = BedFileItem().keys()

group_by='chromosome'

In [5]:
q1 = df1.query('chromosome == \'chr1\'')[['chromosome', 'gene_name', 'start', 'end']]
q2 = df2.query('chromosome == \'chr1\'')[['chromosome', 'gene_name', 'start', 'end']]

In [6]:
rows = PartitionRunner.merge_size(q1, q2, 'chromosome', how='outer')

In [7]:
cols = len(q2.columns) + (len(q1.columns) - 1)

In [8]:
memory = rows * cols * np.dtype(np.float64).itemsize

In [9]:
# B        K       M      G
(((memory / 1024) / 1024) / 1024)

44.880237504839897

In [10]:
slices = 8
q1_slice = q1[:len(q1.index) // slices]
print(len(q1_slice.index))
q2_slice = q2[:len(q2.index) // slices]
print(len(q2_slice.index))

2602
5166


In [11]:
rows = PartitionRunner.merge_size(q1_slice, q2_slice, group_by, how='outer')
print(rows)
memory = rows * cols * np.dtype(np.float64).itemsize
print(memory)
(((memory / 1024) / 1024) / 1024)

13441932
752748192


0.70105138421058655

In [12]:
BYTES_TO_GIGABYTES = 1073741824
MEM_WINDOW = 16

threshold = (psutil.virtual_memory().available / BYTES_TO_GIGABYTES) // MEM_WINDOW
slices = 1
memory = 0
while True:
    rows = PartitionRunner.merge_size(
        q1[:len(q1.index) // slices],
        q2[:len(q2.index) // slices],
        group_by,
        how='outer'
    )
    memory = ((rows * cols * np.dtype(np.float64).itemsize) / BYTES_TO_GIGABYTES)
    if memory < threshold:
        break
    slices += 1

threshold, memory, slices

(2.0, 1.795123279094696, 5)

In [13]:
a = [1, 2, 3]
from itertools import combinations
[c for c in combinations(a, 2)]

[(1, 2), (1, 3), (2, 3)]

In [14]:
merge = q1[:len(q1.index) // slices].merge(
    q2[:len(q2.index) // slices],
    on='chromosome',
    how='outer'
).query('((start_x >= (start_y - 250)) & (end_x <= (end_y + 250))) | ((start_y >= (start_x - 250)) & (end_y <= (end_x + 250)))')

In [15]:
merge

,chromosome,gene_name_x,start_x,end_x,gene_name_y,start_y,end_y
2,chr1,Xkr4,3670345,3672476,Xkr4,3670317,3672272
1735,chr1,Xkr4,3670345,3672476,Xkr4,3670248,3672559
3230,chr1,Xkr4,3670345,3672476,Xkr4,3670899,3671519
24806,chr1,Sox17,4496044,4497086,Sox17,4495974,4496824
26537,chr1,Sox17,4496044,4497086,Sox17,4496225,4497298
28104,chr1,Sox17,4496044,4497086,Sox17,4496603,4497169
51337,chr1,Mrpl15,4774661,4775635,Mrpl15,4774937,4775731
57873,chr1,Mrpl15,4785166,4786129,Mrpl15,4785259,4786102
66141,chr1,Lypla1,4807327,4808638,Lypla1,4807266,4808132
67870,chr1,Lypla1,4807327,4808638,Lypla1,4807430,4808481


In [16]:
pd.DataFrame.from_dict(
    {
        'chromosome': merge['chromosome'],
        'start': merge.loc[:, ['start_x', 'start_y']].min(axis=1),
        'end': merge.loc[:, ['end_x', 'end_y']].max(axis=1)
    }
)

,chromosome,end,start
2,chr1,3672476,3670317
1735,chr1,3672559,3670248
3230,chr1,3672476,3670345
24806,chr1,4497086,4495974
26537,chr1,4497298,4496044
28104,chr1,4497169,4496044
51337,chr1,4775731,4774661
57873,chr1,4786129,4785166
66141,chr1,4808638,4807266
67870,chr1,4808638,4807327


In [17]:
sorted(list(df1.groupby('chromosome').groups.keys()))

['chr1',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chrX',
 'chrY']

In [18]:
BYTES_TO_GIGABYTES = 1073741824
MEM_WINDOW = 16
def get_slices(d1, d2, group_by):
    threshold = (psutil.virtual_memory().available / BYTES_TO_GIGABYTES) // MEM_WINDOW
    slices = 1
    memory = 0
    while True:
        rows = PartitionRunner.merge_size(
            d1[:len(d1.index) // slices],
            d2[:len(d2.index) // slices],
            group_by,
            how='outer'
        )
        memory = ((rows * cols * np.dtype(np.float64).itemsize) / BYTES_TO_GIGABYTES)
        if memory < threshold:
            break
        slices += 1
    return slices

start = time.clock()
groups = df1.groupby(group_by).groups.keys()
for index, group in enumerate(groups):
    sstart = time.clock()
    q1 = df1.query('chromosome == \'{0}\''.format(group))[['chromosome', 'gene_name', 'start', 'end']]
    q2 = df2.query('chromosome == \'{0}\''.format(group))[['chromosome', 'gene_name', 'start', 'end']]
    slices = get_slices(q1, q2, 'chromosome')
    print(
        '---------------------------------- index {0} of {1} ({2} slices) --------------------------------------'.format(
            index, len(groups), slices
        )
    )
    
    merge = q1[:len(q1.index) // slices].merge(
        q2[:len(q2.index) // slices],
        on='chromosome',
        how='outer'
    ).query('((start_x >= (start_y - 250)) & (end_x <= (end_y + 250))) | ((start_y >= (start_x - 250)) & (end_y <= (end_x + 250)))')
    final = pd.DataFrame.from_dict(
        {
            'chromosome': merge['chromosome'],
            'start': merge.loc[:, ['start_x', 'start_y']].min(axis=1),
            'end': merge.loc[:, ['end_x', 'end_y']].max(axis=1)
        }
    )
    send = time.clock()
    print(final)
    print('                                     ------- {0} -------'.format(
            float('{0:.2f}'.format(send - sstart))
        )
    )
end = time.clock()
float('{0:.2f}'.format(end - start))

---------------------------------- index 0 of 21 (3 slices) --------------------------------------
         chromosome       end     start
0             chr17   3116352   3113994
1041          chr17   3116352   3114004
2018          chr17   3116352   3114140
15912         chr17   3315010   3313831
20815         chr17   3558178   3556839
21856         chr17   3558178   3556922
34695         chr17   4908846   4907622
35735         chr17   4909126   4907677
41635         chr17   4994859   4993691
49612         chr17   4997665   4994707
50782         chr17   4997665   4996020
62451         chr17   5190592   5189557
63490         chr17   5190363   5189431
84306         chr17   5440820   5439677
90206         chr17   5493788   5491251
91245         chr17   5493649   5491251
97146         chr17   5795133   5794010
98184         chr17   5795126   5793974
104085        chr17   5799103   5798107
105123        chr17   5799328   5798201
106391        chr17   5799045   5798201
117962        chr17  

         chromosome      end    start
0              chr8  3280302  3278742
1612           chr8  3280074  3278742
5346           chr8  3393858  3392416
6956           chr8  3393858  3392061
10690          chr8  3468048  3466870
12301          chr8  3468056  3467052
16034          chr8  3493809  3492726
17646          chr8  3493661  3492553
21378          chr8  3501190  3500047
22990          chr8  3501559  3499887
26722          chr8  3517523  3515315
28334          chr8  3517789  3515143
32066          chr8  3588758  3587310
33678          chr8  3588364  3587200
35294          chr8  3588364  3587160
37410          chr8  3622052  3620785
39022          chr8  3622026  3620585
42754          chr8  3631729  3630505
44366          chr8  3632503  3630382
53441          chr8  3677389  3675751
55054          chr8  3677389  3675872
58785          chr8  3712324  3711301
67379          chr8  3938403  3937624
80160          chr8  4239561  4238101
81775          chr8  4239563  4238101
85504       

         chromosome       end     start
3             chr18   3271348   3270245
9336          chr18   3338410   3336611
18670         chr18   3383920   3382763
19407         chr18   3383920   3382627
23337         chr18   3509190   3506679
24074         chr18   3509190   3506679
24642         chr18   3509190   3506679
37338         chr18   3769071   3767977
51338         chr18   4353924   4352614
52071         chr18   4353924   4352614
52701         chr18   4353924   4352614
56005         chr18   4376358   4375200
60673         chr18   4440512   4439533
65340         chr18   4635697   4634574
70008         chr18   4657586   4656728
80736         chr18   4795087   4794087
88674         chr18   4922277   4920404
93341         chr18   4924437   4923583
98009         chr18   5099224   5098169
102676        chr18   5125982   5124528
107345        chr18   5335047   5333758
108067        chr18   5335047   5333751
116679        chr18   5593833   5591066
135349        chr18   6137113   6134810


         chromosome       end     start
1              chr9   3259377   3258369
2579           chr9   3259370   3258373
6963           chr9   3336228   3335295
11228          chr9   3404702   3403672
16846          chr9   4139546   4138516
18191          chr9   4139559   4138516
25116          chr9   4219111   4218256
28073          chr9   4310685   4308649
29419          chr9   4310685   4308796
33691          chr9   4796707   4795367
61763          chr9   7177520   7176714
63102          chr9   7177499   7176304
67377          chr9   7185369   7183762
78605          chr9   7765004   7763159
79942          chr9   7764961   7763159
81545          chr9   7764961   7763159
89832          chr9   7837915   7836512
91170          chr9   7837915   7836604
96784          chr9   7873395   7872453
101061         chr9   8005279   8003176
102399         chr9   8005250   8003143
104023         chr9   8005250   8003176
112289         chr9   8134946   8133791
113627         chr9   8134946   8133684


         chromosome       end     start
6618          chr11   3266988   3265522
8559          chr11   3266968   3265522
13236         chr11   3291606   3288316
17362         chr11   3290635   3288316
21795         chr11   3293282   3291778
26472         chr11   3332864   3331548
28413         chr11   3333033   3331409
30599         chr11   3332864   3331761
33091         chr11   3387636   3386742
41654         chr11   3409871   3407564
46326         chr11   3409742   3408734
48271         chr11   3409871   3407564
59561         chr11   3453352   3451620
61506         chr11   3453352   3451425
66179         chr11   3536371   3535030
68127         chr11   3536337   3534961
70320         chr11   3536268   3535281
72797         chr11   3573866   3572781
79415         chr11   3650112   3648136
81364         chr11   3650264   3648145
86034         chr11   3775345   3774320
87984         chr11   3775434   3773872
94603         chr11   3895695   3894421
99270         chr11   3914829   3913043


         chromosome        end      start
2             chr13    3539339    3537370
1042          chr13    3539339    3537321
5109          chr13    3612068    3610075
6149          chr13    3612068    3610075
10216         chr13    3634547    3633607
11256         chr13    3634746    3633607
15327         chr13    3894113    3893044
16363         chr13    3894125    3893044
20434         chr13    3918571    3917531
21470         chr13    3918571    3917531
27669         chr13    4553430    4552597
35758         chr13    5593294    5592232
36793         chr13    5593175    5592253
51080         chr13    5864029    5860104
52116         chr13    5864029    5860104
52117         chr13    5864029    5860104
67439         chr13    6452954    6451527
68713         chr13    6452954    6451589
71509         chr13    6643399    6642291
73844         chr13    6643216    6642273
77652         chr13    6649433    6647640
81730         chr13    8868271    8867331
86837         chr13    8871330    

70.18